- 현재 이 노트북만 이상하게 돌아감 Timeout?
- 일단 Colab에서 돌리는 중

In [4]:
import pandas as pd
import json
from pororo import Pororo
# from pororo.tasks.utils.tokenizer import CustomTokenizer
# SA = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')

# pororo 라벨링, 1차 단어사전 만들기

In [1]:
import pandas as pd
import json
from pororo import Pororo
import os
from tqdm import tqdm

s_nm = Pororo(task = 'sentiment',model = 'brainbert.base.ko.nsmc', lang = 'ko')
s_sh = Pororo(task = 'sentiment', model = 'brainbert.base.ko.shopping', lang = 'ko')
DC = pd.DataFrame(columns = ['_id', 'code', 'date', 'title', 'Rec', 'View', 'nsmc', 'shop'])

for f in os.listdir('../preprocess'):
    if f.endswith('json') and f.startswith('DC'):
        for line in tqdm(open('../preprocess/'+f, 'r', encoding = 'utf8').readlines()):
            j = json.loads(line)
            nm_sa = s_nm(j['title'], show_probs = True)['positive']
            sh_sa = s_sh(j['title'], show_probs = True)['positive']
            j.update(nsmc = nm_sa, shop = sh_sa)
            DC.append(j, ignore_index = True)
            DC.to_pickle('./dc_all.pkl')

In [2]:
import pickle
import pandas as pd

In [3]:
NAVER = pd.read_pickle('./naver_all.pkl')
NAVER.head()

,_id,code,date,title,views,pos,neg,nsmc,shop
0,{'$oid': '61886de6bdfa455b98338f62'},008770,2021.11.08 09:22,개미털기 각입니다,4,1,0,0.014268,0.030229
1,{'$oid': '61886de6bdfa455b98338f63'},008770,2021.11.08 09:22,얘들야 이러다 다죽어,2,0,0,0.025835,0.094668
2,{'$oid': '61886de6bdfa455b98338f64'},008770,2021.11.08 09:22,종합지수가,3,0,0,0.167821,0.145982
3,{'$oid': '61886de6bdfa455b98338f65'},008770,2021.11.08 09:21,일단 차익실현,10,0,0,0.023304,0.214951
4,{'$oid': '61886de6bdfa455b98338f66'},008770,2021.11.08 09:19,공매들 매도에서 매수로 손 바뀜 이 일어 낫네요.,34,1,0,0.005306,0.025457


In [5]:
# 중복 제거
NAVER.drop_duplicates(subset = ['code', 'title'], inplace=True)
# NAVER[NAVER.duplicated(['code', 'title'])]

In [6]:
NAVER['positive'] = (NAVER['nsmc'] + NAVER.shop) / 2
NAVER.head()

,_id,code,date,title,views,pos,neg,nsmc,shop,positive
0,{'$oid': '61886de6bdfa455b98338f62'},008770,2021.11.08 09:22,개미털기 각입니다,4,1,0,0.014268,0.030229,0.022248
1,{'$oid': '61886de6bdfa455b98338f63'},008770,2021.11.08 09:22,얘들야 이러다 다죽어,2,0,0,0.025835,0.094668,0.060252
2,{'$oid': '61886de6bdfa455b98338f64'},008770,2021.11.08 09:22,종합지수가,3,0,0,0.167821,0.145982,0.156901
3,{'$oid': '61886de6bdfa455b98338f65'},008770,2021.11.08 09:21,일단 차익실현,10,0,0,0.023304,0.214951,0.119127
4,{'$oid': '61886de6bdfa455b98338f66'},008770,2021.11.08 09:19,공매들 매도에서 매수로 손 바뀜 이 일어 낫네요.,34,1,0,0.005306,0.025457,0.015382


In [14]:
from konlpy.tag import Okt, Hannanum, Kkma#, Mecab
han = Hannanum()
okt = Okt()
kkma = Kkma()
# mecab = Mecab()


In [16]:
NAVER['hannanum'] = NAVER.title.apply(lambda x: han.pos(x))
NAVER['okt'] = NAVER.title.apply(lambda x: okt.pos(x))
NAVER['kkma'] = NAVER.title.apply(lambda x: kkma.pos(x))

In [17]:
NAVER

,_id,code,date,title,views,pos,neg,nsmc,shop,positive,hannanum,okt,kkma
0,{'$oid': '61886de6bdfa455b98338f62'},008770,2021.11.08 09:22,개미털기 각입니다,4,1,0,0.014268,0.030229,0.022248,"[(개미털기, N), (각, N), (이, J), (ㅂ니다, E)]","[(개미, Noun), (털기, Verb), (각, Noun), (입니다, Adje...","[(개미, NNG), (털, NNG), (기, XSN), (각, NNG), (이, ..."
1,{'$oid': '61886de6bdfa455b98338f63'},008770,2021.11.08 09:22,얘들야 이러다 다죽어,2,0,0,0.025835,0.094668,0.060252,"[(얘들, N), (야, J), (이르, P), (어다, E), (다죽어, N)]","[(얘, Noun), (들, Suffix), (야, Josa), (이러다, Verb...","[(얘, UN), (들, VV), (ㄹ, ETD), (야, NNG), (이러, VV..."
2,{'$oid': '61886de6bdfa455b98338f64'},008770,2021.11.08 09:22,종합지수가,3,0,0,0.167821,0.145982,0.156901,"[(종합지수, N), (가, J)]","[(종합, Noun), (지수, Noun), (가, Josa)]","[(종합, NNG), (지수, NNG), (가, JKS)]"
3,{'$oid': '61886de6bdfa455b98338f65'},008770,2021.11.08 09:21,일단 차익실현,10,0,0,0.023304,0.214951,0.119127,"[(일단, N), (차익실현, N)]","[(일단, Noun), (차익, Noun), (실현, Noun)]","[(일단, NNG), (차익, NNG), (실현, NNG)]"
4,{'$oid': '61886de6bdfa455b98338f66'},008770,2021.11.08 09:19,공매들 매도에서 매수로 손 바뀜 이 일어 낫네요.,34,1,0,0.005306,0.025457,0.015382,"[(공매들, N), (매도, N), (에서, J), (매수, N), (로, J), ...","[(공매, Noun), (들, Suffix), (매도, Noun), (에서, Jos...","[(공매, NNG), (들, XSN), (매도, NNG), (에서, JKM), (매..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11431,{'$oid': '618870babdfa455b9833bc09'},009830,2021.08.31 14:55,족되따 싶다,184,1,1,0.070234,0.636528,0.353381,"[(족되따, N), (싶, P), (다, E)]","[(족, Noun), (되, Verb), (따, Verb), (싶다, Verb)]","[(족되, UN), (따, VV), (아, ECS), (싶, VXA), (다, EFN)]"
11432,{'$oid': '618870babdfa455b9833bc0a'},009830,2021.08.31 14:35,목표가가 다들 어디십니까?,569,1,1,0.746721,0.022196,0.384459,"[(목표가가, N), (다들, N), (어디, N), (이, J), (시ㅂ니까, E...","[(목표, Noun), (가가, Noun), (다, Adverb), (들, Verb...","[(목표, NNG), (가, JKS), (가, VV), (아, ECS), (다, N..."
11434,{'$oid': '618870babdfa455b9833bc0c'},009830,2021.08.31 14:18,팍팍올라가는거 기다리지말고 천천히 혀,183,2,1,0.906076,0.179329,0.542703,"[(팍팍올라가는거, N), (기다리, P), (지, E), (말, P), (고, E...","[(팍팍, Adverb), (올라가는거, Verb), (기다리지말고, Verb), ...","[(팍팍, MAG), (올라가, VV), (는, ETD), (거, NNB), (기다..."
11435,{'$oid': '618870babdfa455b9833bc0d'},009830,2021.08.31 14:18,외인이 이렇게 많이 사는데,216,3,0,0.235619,0.218866,0.227243,"[(외인, N), (이, J), (이렇, P), (게, E), (많, P), (이,...","[(외인, Noun), (이, Josa), (이렇게, Adverb), (많이, Ad...","[(외인, NNG), (이, JKS), (이렇게, MAG), (많이, MAG), (..."
